# Data Wrangling: Join, Combine, and Reshape

In [3]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Hierarchical Indexing

### Series

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
# select b elements
# (1) just ['b']
# (2) .loc
# (3) .iloc

data['b']

1   -0.555730
3    1.965781
dtype: float64

In [5]:
data.loc['b']

1   -0.555730
3    1.965781
dtype: float64

In [8]:
data.iloc[3:5]

b  1   -0.555730
   3    1.965781
dtype: float64

### DataFrame

In [4]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
# select 'Ohio' columns
# (1) frame[]
# (2) frame.loc[:,]

frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [18]:
frame.loc[:, 'Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [ ]:
# select 'a' rows
# frame.loc

In [5]:
frame[:'a']

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5

### Reordering and Sorting Levels

In [8]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [ ]:
# swaplevel(,)
# swaplevel(,,axis=1)

In [11]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [15]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [17]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [20]:
frame.swaplevel('state','color')

KeyError: 'Level state not found'

### Summary Statistics by Level

In [21]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [22]:
frame.sum()

state     color
Ohio      Green    18
          Red      22
Colorado  Green    26
dtype: int64

In [23]:
frame.sum(axis=1)

key1  key2
a     1        3
      2       12
b     1       21
      2       30
dtype: int64

In [24]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [25]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns

In [26]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [ ]:
# change the columns c, d as index: set_index()
# back to index c, d to columns: reset_index()

## Combining and Merging Datasets

### Database-Style DataFrame Joins

In [33]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [34]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [ ]:
# combine df1, df2
# merge( , , on='key', how='') # inner, left, right, outer

In [35]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [36]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [37]:
pd.merge(df1,df2,on='key',how='right')

,key,data1,data2
0,b,0.0,1
1,b,1.0,1
2,b,6.0,1
3,a,2.0,0
4,a,4.0,0
5,a,5.0,0
6,d,NaN,2


In [38]:
pd.merge(df1,df2,on='key',how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [ ]:
# merge on 'key1', 'key2'


In [39]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [40]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [41]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [44]:
pd.merge(left,right,on=['key1','key2'])

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


### Merging on Index

In [25]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])


In [26]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [27]:
right1

,group_val
a,3.5
b,7.0


In [31]:
# combine left1 and right1,  left_on='key', right_index=True, how='left'
# (1) merge
# (2) join

In [32]:
frame.set_index(['c','d'])

KeyError: "None of ['c', 'd'] are in the columns"

In [45]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])


In [46]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [47]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [ ]:
# combine lefth and righth, left_on=['key','key2'], right_index=True, how='left'
# (1) merge
# (2) join

In [48]:
pd.merge(left1,right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [49]:
left1.join(right1, on='key',how='inner')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [51]:
left1.join(right1, on='key',how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


### Concatenating Along an Axis

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [ ]:
s1

In [ ]:
s2

In [ ]:
s3

In [ ]:
# conat for series
# (1) pd.concat[s1, s2, s3]
# (2) for axis=1

In [ ]:
# concat for DataFrame 

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [ ]:
df1

In [ ]:
df2

### Combining Data with Overlap

In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

In [ ]:
a

In [ ]:
b

In [ ]:
a.combine_first(b)

## Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

In [64]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [65]:
# try stack() and unstack()

In [66]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [67]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [68]:
s2

c    4
d    5
e    6
dtype: int64

In [69]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [ ]:
# unstack


In [ ]:
# unstack().stack()


### Pivoting “Long” to “Wide” Format, Pivoting “Wide” to “Long” Format

In [52]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [ ]:
# (1) melt id_vars=['key']


In [54]:
# pivot('key', 'variable', 'value')
melted.pivot('key','variavle','value')

NameError: name 'melted' is not defined

In [ ]:
reshaped.reset_index()

In [ ]:
# melt on ['key']
melted = pd.melt(reshaped.reset_index(), ['key'])
melted

## another example

In [55]:
df = pd.DataFrame({'key': ['b', 'a', 'b'],
                   'data1': range(3)})
df

,key,data1
0,b,0
1,a,1
2,b,2


In [57]:
dummies = pd.get_dummies(df['key'])
dummies

,a,b
0,0,1
1,1,0
2,0,1


In [ ]:
# (1) use join
# (2) use concat

In [59]:
pd.DataFrame(df['data1']).join(dummies)

,data1,a,b
0,0,0,1
1,1,1,0
2,2,0,1


In [70]:
pd.concat([df['data1'],dummies],axis =1)

,data1,a,b
0,0,0,1
1,1,1,0
2,2,0,1


In [71]:
pd.concat([df,dummies])

,key,data1,a,b
0,b,0.0,NaN,NaN
1,a,1.0,NaN,NaN
2,b,2.0,NaN,NaN
0,NaN,NaN,0.0,1.0
1,NaN,NaN,1.0,0.0
2,NaN,NaN,0.0,1.0
